In [1]:
!pip install vllm==0.8.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 MB 24.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 10.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 138.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 146.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━

In [17]:
import pandas as pd
from datasets import Dataset
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

In [9]:
test_df = pd.read_excel('테스트 셋_800 완료.xlsx')

In [10]:
test_df.head()

,관리번호,설비번호(구),설비명,수리일자,고장내용,고장부품,불량유형,조치내용,수리처,수리시간(분),수리금액
0,Z000000001,Z000000001,기타,2023-01-03,3공장렉 철거후 2공장으로 이동,3공장 렉,철거,2공장으로 이동,오정수이승희,480,NaN
1,Z000000001,Z000000001,기타,2023-01-03,CN7라인 천정 전등 교체,CN7라인 천정 전등,고장,전등 교체,송명호윤동태,180,NaN
2,Z000000001,Z000000001,기타,2023-01-03,E4L 인너카바 리벳툴 진공이상 리벳 잔영물 제거 청소,E4L 인너카바 리벳툴,진공이상,리벳 잔영물 제거 청소,최혁진윤동태,30,NaN
3,Z000000001,Z000000001,기타,2023-01-03,전착도장 대차 제작중,전착도장 대차,NaN,제작중,김정표오정수,140,NaN
4,M000000038,ASM-029,조립기(DH 스위치 조립기),2023-01-04,#40 LH 트램스파 동작이상 전후진 솔밸브 단선 수리,#40 LH 트램스파 전후진 솔밸브,동작이상,단선 수리,송명호윤동태,150,NaN


In [11]:
def format_data(sample):
    # 고장부품, 불량유형, 조치내용을 딕셔너리 형태로 구성
    response_dict = {
        "고장부품": sample["고장부품"],
        "불량유형": sample["불량유형"], 
        "조치내용": sample["조치내용"]
    }
    
    return {
        "messages": [
            {
                "role": "system",
                "content": '''주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.
이때 파이썬의 Dictionary 형태로 반환하시오.''',
            },
            {
                "role": "user",
                "content": f"고장 내용: {sample['고장내용']}",
            },
            {
                "role": "assistant",
                "content": str(response_dict)  # 나머지 3개 필드를 딕셔너리로 변환하여 assistant 응답으로
            },
        ],
    }

In [12]:
test_dataset = []
for _, row in test_df.iterrows():
    test_dataset.append(format_data(row))

print(f"\n테스트 데이터 개수: Test {len(test_dataset)}개")


테스트 데이터 개수: Test 800개


In [13]:
test_dataset = Dataset.from_list(test_dataset)

In [18]:
base_model_id = "iamjoon/Qwen-StructureParser"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

In [23]:
prompt_lst = []
label_lst = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label.replace('<|im_end|>', '').strip())

In [24]:
print(prompt_lst[42])

<|im_start|>system
주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.
이때 파이썬의 Dictionary 형태로 반환하시오.<|im_end|>
<|im_start|>user
고장 내용: 생산수량 간헐적 수량 누락 카운터 단자 재 조임<|im_end|>
<|im_start|>assistant


In [25]:
print(label_lst[42])

{'고장부품': '카운터 단자', '불량유형': '생산 수량 간헐적 누락', '조치내용': '카운터 단자 재 조임'}


In [26]:
llm = LLM(model="iamjoon/Qwen-StructureParser")

INFO 06-24 08:04:12 [config.py:585] This model supports multiple tasks: {'score', 'reward', 'classify', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 06-24 08:04:12 [config.py:1697] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 06-24 08:04:14 [core.py:54] Initializing a V1 LLM engine (v0.8.2) with config: model='iamjoon/Qwen-StructureParser', speculative_config=None, tokenizer='iamjoon/Qwen-StructureParser', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpo

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 06-24 08:04:17 [loader.py:447] Loading weights took 0.29 seconds
INFO 06-24 08:04:17 [gpu_model_runner.py:1186] Model loading took 0.9266 GB and 0.996923 seconds
INFO 06-24 08:04:22 [backends.py:415] Using cache directory: /root/.cache/vllm/torch_compile_cache/c381a883d3/rank_0_0 for vLLM's torch.compile
INFO 06-24 08:04:22 [backends.py:425] Dynamo bytecode transform time: 5.41 s
INFO 06-24 08:04:26 [backends.py:132] Cache the graph of shape None for later use
INFO 06-24 08:04:47 [backends.py:144] Compiling a graph for general shape takes 24.42 s
INFO 06-24 08:04:56 [monitor.py:33] torch.compile takes 29.83 s in total
INFO 06-24 08:04:57 [kv_cache_utils.py:566] GPU KV cache size: 5,223,792 tokens
INFO 06-24 08:04:57 [kv_cache_utils.py:569] Maximum concurrency for 32,768 tokens per request: 159.42x
INFO 06-24 08:05:20 [gpu_model_runner.py:1534] Graph capturing finished in 23 secs, took 0.38 GiB
INFO 06-24 08:05:21 [core.py:151] init engine (profile, create kv cache, warmup model) t

In [28]:
# 샘플링 파라미터 설정
sampling_params = SamplingParams(
    temperature=0,
    max_tokens=1024,
    stop=["<|im_end|>"],  # 문자열로 중지 토큰 지정
)

In [29]:
outputs = llm.generate(prompt_lst, sampling_params)

Processed prompts: 100%|██████████| 800/800 [00:02<00:00, 378.47it/s, est. speed input: 30121.44 toks/s, output: 17672.87 toks/s]


In [35]:
preds = [pred.outputs[0].text.strip() for pred in outputs]

In [36]:
test_df['predictions'] = preds
test_df['labels'] = label_lst

In [38]:
test_df.to_csv('evaluation.csv', index=False)

In [39]:
test_df

,관리번호,설비번호(구),설비명,수리일자,고장내용,고장부품,불량유형,조치내용,수리처,수리시간(분),수리금액,predictions,labels
0,Z000000001,Z000000001,기타,2023-01-03,3공장렉 철거후 2공장으로 이동,3공장 렉,철거,2공장으로 이동,오정수이승희,480,NaN,"{'고장부품': '3공장렉', '불량유형': '철거', '조치내용': '2공장으로 ...","{'고장부품': '3공장 렉', '불량유형': '철거', '조치내용': '2공장으로..."
1,Z000000001,Z000000001,기타,2023-01-03,CN7라인 천정 전등 교체,CN7라인 천정 전등,고장,전등 교체,송명호윤동태,180,NaN,"{'고장부품': 'CN7라인 천정 전등', '불량유형': '고장', '조치내용': ...","{'고장부품': 'CN7라인 천정 전등', '불량유형': '고장', '조치내용': ..."
2,Z000000001,Z000000001,기타,2023-01-03,E4L 인너카바 리벳툴 진공이상 리벳 잔영물 제거 청소,E4L 인너카바 리벳툴,진공이상,리벳 잔영물 제거 청소,최혁진윤동태,30,NaN,"{'고장부품': 'E4L 인너카바 리벳툴', '불량유형': '진공이상', '조치내용...","{'고장부품': 'E4L 인너카바 리벳툴', '불량유형': '진공이상', '조치내용..."
3,Z000000001,Z000000001,기타,2023-01-03,전착도장 대차 제작중,전착도장 대차,NaN,제작중,김정표오정수,140,NaN,"{'고장부품': '전착도장', '불량유형': '제작 중', '조치내용': '제작 중'}","{'고장부품': '전착도장 대차', '불량유형': nan, '조치내용': '제작중'}"
4,M000000038,ASM-029,조립기(DH 스위치 조립기),2023-01-04,#40 LH 트램스파 동작이상 전후진 솔밸브 단선 수리,#40 LH 트램스파 전후진 솔밸브,동작이상,단선 수리,송명호윤동태,150,NaN,"{'고장부품': '#40 LH 트램스파', '불량유형': '동작이상', '조치내용'...","{'고장부품': '#40 LH 트램스파 전후진 솔밸브', '불량유형': '동작이상'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,Z000000001,Z000000001,기타,2023-06-03,포르비아시팅 라인 이관 작업,포르비아시팅 라인,이관 작업,이관 작업 진행,정은호오정수,300,NaN,"{'고장부품': '포르비아시팅 라인', '불량유형': '이관', '조치내용': '이...","{'고장부품': '포르비아시팅 라인', '불량유형': '이관 작업', '조치내용':..."
796,S000000019,CNV-001,스크랩 컨베어,2023-06-05,"구동불가 주모터 반대 스크랩제거,아웃플레이트 1EA 설치,샤프트분할핀 파손교체 다량","주모터, 아웃플레이트, 샤프트분할핀","스크랩 이물질 결함, 아웃플레이트 누락, 샤프트분할핀 파손","스크랩 제거, 아웃플레이트 1EA 설치, 샤프트분할핀 교체",생기전원,200,NaN,"{'고장부품': '주모터 반대 스크랩, 샤프트 분할핀', '불량유형': '구동불가,...","{'고장부품': '주모터, 아웃플레이트, 샤프트분할핀', '불량유형': '스크랩 이..."
797,Z000000001,Z000000001,기타,2023-06-05,E4L 공박스 청소라인 선풍기 2대 설치,E4L 공박스 청소라인,설치 필요,선풍기 2대 설치,윤동탸오정수,100,NaN,"{'고장부품': 'E4L 공박스 청소라인 선풍기', '불량유형': '설치 불량', ...","{'고장부품': 'E4L 공박스 청소라인', '불량유형': '설치 필요', '조치내..."
798,Z000000001,Z000000001,기타,2023-06-05,수동리프트 발판 파손 용접 작업,수동리프트 발판,파손,용접 작업으로 복원,오정수윤동태,50,NaN,"{'고장부품': '수동리프트 발판', '불량유형': '파손', '조치내용': '용접...","{'고장부품': '수동리프트 발판', '불량유형': '파손', '조치내용': '용접..."
